In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.model_selection import train_test_split




In [ ]:
df = pd.read_csv('IPL_dat.csv')

df.drop(columns=['MatchID','Defending Team','Chasing Team'], inplace=True)


df_encoded = pd.get_dummies(df, columns=['Defending Stadium'], dtype=int)

df_encoded['Defending Toss'] = df['Defending Toss'].apply(lambda x: 1 if x == 'WON' else 0)
df_encoded['Time'] = df['Time'].apply(lambda x: 1 if x == 'Day' else 0)

column_names = df_encoded.columns.tolist()

do_normalize = ["Bat2 Strength", "Bowl2 Strength", "First Inn Score", "First Inn Median",
                "Second Inn Median", "Ground Altitude", "Ground Length", "Ground Width"]

column_names = df_encoded.columns.tolist()

dont_normalize = [name for name in column_names if name not in do_normalize]

normalized_data = df_encoded[do_normalize]
unchanged_data = df_encoded[dont_normalize]

normalization_layer = Normalization()
normalization_layer.adapt(normalized_data)

normalized_columns = normalization_layer(normalized_data)

normalized_columns_df = pd.DataFrame(normalized_columns.numpy(), columns=do_normalize)

data = pd.concat([normalized_columns_df, unchanged_data], axis=1)



defending_result = data.pop('Defending Result')
data.insert(0, 'Defending Result', defending_result)
#print(tabulate(data, headers='keys', tablefmt='psql'))

X = data[data.columns[:-1]].values
y = data[data.columns[-1]].values

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'), # if x <= 0 --> 0, x > 0 --> x
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_valid, y_valid))

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
28/28 [==============================] - 2s 6ms/step - loss: 0.6654 - accuracy: 0.6157
Epoch 2/50
28/28 [==============================] - 0s 5ms/step - loss: 0.5560 - accuracy: 0.7506
Epoch 3/50
28/28 [==============================] - 0s 7ms/step - loss: 0.4896 - accuracy: 0.7663
Epoch 4/50
28/28 [==============================] - 0s 6ms/step - loss: 0.4397 - accuracy: 0.7955
Epoch 5/50
28/28 [==============================] - 0s 5ms/step - loss: 0.3917 - accuracy: 0.8404
Epoch 6/50
28/28 [==============================] - 0s 3ms/step - loss: 0.3434 - accuracy: 0.8809
Epoch 7/50
28/28 [==============================] - 0s 2ms/step - loss: 0.3029 - accuracy: 0.9146
Epoch 8/50
28/28 [==============================] - 0s 3ms/step - loss: 0.2665 - accuracy: 0.9236
Epoch 9/50
28/28 [==============================] - 0s 3ms/step - loss: 0.2376 - accuracy: 0.9348
Epoch 10/50
28/28 [==============================] - 0s 3ms/step - loss: 0.2133 - accuracy: 0.9348
Epoch 11/50
28/28 [